In [3]:
import os
import glob
import pandas as pd
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt


In [4]:
def compute_shannon_entropy(values, bins=10, epsilon=1e-10):
    """
    Compute Shannon entropy for a set of values by binning them.
    Adds a tiny constant (epsilon) to every bin to preserve zeros (which may represent sleep).
    """
    counts, _ = np.histogram(values, bins=bins)
    # Add epsilon to each bin count so that 0's become epsilon
    counts = counts + epsilon
    total = counts.sum()
    if total > 0:
        probabilities = counts / total
        # (No need to filter zeros since we replaced them)
        return entropy(probabilities, base=2)
    else:
        return np.nan


In [5]:
# Read one subject's CSV file (for testing)
df = pd.read_csv("\\\\itf-rs-store24.hpc.uiowa.edu\\vosslabhpc\\symposia\\cpsy-25\\temp\\GGIR\\sub-11\\output_sub-207\\meta\\csv\\207_(2015-07-08)RAW.csv.RData.csv")

# Convert the 'timestamp' column to datetime and create a 'date' column (floored to midnight)
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['date'] = df['timestamp'].dt.floor('D')

# Optional: inspect the unique dates to ensure the splicing is correct.
print("Unique dates in the data:", df['date'].unique())

# Aggregate ENMO for each day by computing daily Shannon entropy
daily_entropy = df.groupby('date').apply(
    lambda group: pd.Series({
        'time_interval': f"{group.name} to {group.name + pd.Timedelta(days=1)}",
        'shannon_entropy': compute_shannon_entropy(group['ENMO'], bins=10)
    })
).reset_index()

print("Daily summary:")
print(daily_entropy)


Unique dates in the data: <DatetimeArray>
['2015-07-08 00:00:00+00:00', '2015-07-09 00:00:00+00:00',
 '2015-07-10 00:00:00+00:00', '2015-07-11 00:00:00+00:00',
 '2015-07-12 00:00:00+00:00', '2015-07-13 00:00:00+00:00',
 '2015-07-14 00:00:00+00:00', '2015-07-15 00:00:00+00:00']
Length: 8, dtype: datetime64[ns, UTC]
Daily summary:
                       date  \
0 2015-07-08 00:00:00+00:00   
1 2015-07-09 00:00:00+00:00   
2 2015-07-10 00:00:00+00:00   
3 2015-07-11 00:00:00+00:00   
4 2015-07-12 00:00:00+00:00   
5 2015-07-13 00:00:00+00:00   
6 2015-07-14 00:00:00+00:00   
7 2015-07-15 00:00:00+00:00   

                                       time_interval  shannon_entropy  
0  2015-07-08 00:00:00+00:00 to 2015-07-09 00:00:...         1.102900  
1  2015-07-09 00:00:00+00:00 to 2015-07-10 00:00:...         1.094347  
2  2015-07-10 00:00:00+00:00 to 2015-07-11 00:00:...         0.420170  
3  2015-07-11 00:00:00+00:00 to 2015-07-12 00:00:...         0.594309  
4  2015-07-12 00:00:00+00:00 

C:\Users\jedim\AppData\Local\Temp\ipykernel_2556\522852623.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_entropy = df.groupby('date').apply(


In [6]:
# Use the daily entropy values to compute meta entropy for this subject
n_bins = min(10, len(daily_entropy))
subject_meta_entropy = compute_shannon_entropy(daily_entropy['shannon_entropy'], bins=n_bins)

print("Subject Meta Entropy:", subject_meta_entropy)


Subject Meta Entropy: 2.500000000891549


In [7]:
import os
import glob
import pandas as pd
import numpy as np
from scipy.stats import entropy

# Define glob patterns for each study using raw strings
better_pattern = "\\\\itf-rs-store24.hpc.uiowa.edu\\vosslabhpc\\Projects\\BETTER\\3-Experiment\\2-data\\bids\\derivatives\\GGIR\\sub-GE*\\ses-pre\\beh\\output_beh\\meta\\csv\\*.csv.RData.csv"
extend_pattern = "\\\\itf-rs-store24.hpc.uiowa.edu\\vosslabhpc\\Projects\\BikeExtend\\3-Experiment\\2-Data\\BIDS\\derivatives\\GGIR_2.8.2\\sub-*\\ses-*/beh\\output_beh\\meta\\csv\\*.csv.RData.csv"
pacrd_pattern = "\\\\itf-rs-store24.hpc.uiowa.edu\\vosslabhpc\\symposia\\cpsy-25\\temp\\GGIR\\sub-*\\output_sub-*/meta\\csv\\*.csv.RData.csv"

# Find file paths for each study
better_files = glob.glob(better_pattern)
extend_files = glob.glob(extend_pattern)
pacrd_files  = glob.glob(pacrd_pattern)

print("BETTER files found:", len(better_files))
print("EXTEND files found:", len(extend_files))
print("PACRD files found:", len(pacrd_files))


def compute_shannon_entropy(values, bins=10, epsilon=1e-10):
    """
    Compute Shannon entropy for a set of values by binning them.
    Adds a tiny constant (epsilon) to each bin so that 0's become a very small number.
    """
    counts, _ = np.histogram(values, bins=bins)
    counts = counts + epsilon
    total = counts.sum()
    if total > 0:
        probabilities = counts / total
        return entropy(probabilities, base=2)
    else:
        return np.nan


def process_subject(csv_path, bins=10):
    """
    Process a subject's CSV file and return the daily summary and meta entropy.
    Meta entropy is computed as the Shannon entropy of the normalized daily entropy values.
    """
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
        return None, None

    # Check if the 'timestamp' column exists
    if 'timestamp' not in df.columns:
        print(f"'timestamp' column not found in {csv_path}, skipping.")
        return None, None

    # Parse timestamps; unparseable values become NaT
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)
    
    # Skip file if all timestamps failed to parse
    if df['timestamp'].isna().all():
        print(f"All timestamps in {csv_path} could not be parsed, skipping.")
        return None, None
    
    # Drop rows with NaT timestamps
    df = df.dropna(subset=['timestamp'])
    
    # Floor the timestamp to create a 'date' column (grouping by day)
    if pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df['date'] = df['timestamp'].dt.floor('D')
    else:
        print(f"Timestamp column in {csv_path} is not datetime after parsing, skipping.")
        return None, None

    # Group by the floored date and compute daily Shannon entropy of the ENMO values
    daily_summary = df.groupby('date', group_keys=False).apply(
        lambda group: pd.Series({
            'time_interval': f"{group.name} to {group.name + pd.Timedelta(days=1)}",
            'shannon_entropy': compute_shannon_entropy(group['ENMO'], bins=bins)
        })
    ).reset_index()

    # Calculate meta entropy: normalize daily entropy values then compute entropy of that distribution
    daily_entropy_values = daily_summary['shannon_entropy'].values
    if daily_entropy_values.sum() > 0:
        meta_ent = entropy(daily_entropy_values / daily_entropy_values.sum(), base=2)
    else:
        meta_ent = np.nan

    return daily_summary, meta_ent


# A helper function to extract a subject identifier from the file path
def get_subject_id(file_path):
    # You can customize this extraction logic as needed.
    return os.path.basename(file_path)


# Create a list to store summary information for each subject
subject_summary = []

# Process subjects for each study and collect meta entropy values.
for study, file_list in [("BETTER", better_files), ("EXTEND", extend_files), ("PACRD", pacrd_files)]:
    for file in file_list:
        print(f"Processing {study} file: {file}")
        _, meta_ent = process_subject(file)
        if meta_ent is not None:
            subject_id = get_subject_id(file)
            subject_summary.append({
                "study": study,
                "subject_id": subject_id,
                "meta_entropy": meta_ent
            })

# Convert the summary list into a DataFrame
summary_df = pd.DataFrame(subject_summary)

# Display the summary DataFrame
print(summary_df.head())

# Optionally, save the summary DataFrame to a CSV file
summary_df.to_csv("meta_entropy_summary_all_subjects.csv", index=False)
print("Saved summary to meta_entropy_summary_all_subjects.csv")



BETTER files found: 65
EXTEND files found: 681
PACRD files found: 71
Processing BETTER file: \\itf-rs-store24.hpc.uiowa.edu\vosslabhpc\Projects\BETTER\3-Experiment\2-data\bids\derivatives\GGIR\sub-GE120001\ses-pre\beh\output_beh\meta\csv\sub-GE120001_ses-pre_accel.csv.RData.csv


C:\Users\jedim\AppData\Local\Temp\ipykernel_2556\4078523351.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_summary = df.groupby('date', group_keys=False).apply(


Processing BETTER file: \\itf-rs-store24.hpc.uiowa.edu\vosslabhpc\Projects\BETTER\3-Experiment\2-data\bids\derivatives\GGIR\sub-GE120002\ses-pre\beh\output_beh\meta\csv\sub-GE120002_ses-pre_accel.csv.RData.csv


C:\Users\jedim\AppData\Local\Temp\ipykernel_2556\4078523351.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_summary = df.groupby('date', group_keys=False).apply(


Processing BETTER file: \\itf-rs-store24.hpc.uiowa.edu\vosslabhpc\Projects\BETTER\3-Experiment\2-data\bids\derivatives\GGIR\sub-GE120003\ses-pre\beh\output_beh\meta\csv\sub-GE120003_ses-pre_accel.csv.RData.csv


...

C:\Users\jedim\AppData\Local\Temp\ipykernel_2556\4078523351.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_summary = df.groupby('date', group_keys=False).apply(


Processing PACRD file: \\itf-rs-store24.hpc.uiowa.edu\vosslabhpc\symposia\cpsy-25\temp\GGIR\sub-92\output_sub-547\meta\csv\547_(2015-11-24)RAW.csv.RData.csv


C:\Users\jedim\AppData\Local\Temp\ipykernel_2556\4078523351.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_summary = df.groupby('date', group_keys=False).apply(


Processing PACRD file: \\itf-rs-store24.hpc.uiowa.edu\vosslabhpc\symposia\cpsy-25\temp\GGIR\sub-98\output_sub-552\meta\csv\552_(2015-12-16)RAW.csv.RData.csv
    study                                subject_id  meta_entropy
0  BETTER  sub-GE120001_ses-pre_accel.csv.RData.csv      2.955153
1  BETTER  sub-GE120002_ses-pre_accel.csv.RData.csv      2.924345
2  BETTER  sub-GE120003_ses-pre_accel.csv.RData.csv      2.853231
3  BETTER  sub-GE120004_ses-pre_accel.csv.RData.csv      2.889990
4  BETTER  sub-GE120005_ses-pre_accel.csv.RData.csv      2.888399
Saved summary to meta_entropy_summary_all_subjects.csv


C:\Users\jedim\AppData\Local\Temp\ipykernel_2556\4078523351.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_summary = df.groupby('date', group_keys=False).apply(


In [ ]:
import matplotlib.pyplot as plt

# Plot distribution for each study
studies = summary_df['study'].unique()
for study in studies:
    study_data = summary_df[summary_df['study'] == study]['meta_entropy'].dropna()
    plt.figure(figsize=(8,5))
    plt.hist(study_data, bins=10, alpha=0.75, edgecolor='black')
    plt.title(f"Meta Entropy Distribution - {study} Study")
    plt.xlabel("Meta Entropy")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

# Plot distribution for all studies combined
all_data = summary_df['meta_entropy'].dropna()
plt.figure(figsize=(8,5))
plt.hist(all_data, bins=10, alpha=0.75, edgecolor='black')
plt.title("Meta Entropy Distribution - All Studies Combined")
plt.xlabel("Meta Entropy")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


NameError: name 'meta_entropies' is not defined